In [1]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=778d777c445a38f9d895c4e28b81afbc567a88ad6a5b20208d86749320fbf2df
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [4]:
data = spark.read.csv('seeds_dataset.csv',inferSchema=True,header=True)

In [5]:
data.show()

+-----+---------+-----------+------------------+------------------+---------------------+------------------+
| area|perimeter|compactness|  length_of_kernel|   width_of_kernel|asymmetry_coefficient|  length_of_groove|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+
|15.26|    14.84|      0.871|             5.763|             3.312|                2.221|              5.22|
|14.88|    14.57|     0.8811| 5.553999999999999|             3.333|                1.018|             4.956|
|14.29|    14.09|      0.905|             5.291|3.3369999999999997|                2.699|             4.825|
|13.84|    13.94|     0.8955|             5.324|3.3789999999999996|                2.259|             4.805|
|16.14|    14.99|     0.9034|5.6579999999999995|             3.562|                1.355|             5.175|
|14.38|    14.21|     0.8951|             5.386|             3.312|   2.4619999999999997|             4.956|
|14.69|    14.49|  

In [8]:
data.columns

['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [6]:
from pyspark.ml.clustering import KMeans

In [7]:
from pyspark.ml.feature import VectorAssembler

In [10]:
assembler = VectorAssembler(inputCols=data.columns,outputCol='features')

In [11]:
final  = assembler.transform(data)

In [12]:
final.show()

+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+
| area|perimeter|compactness|  length_of_kernel|   width_of_kernel|asymmetry_coefficient|  length_of_groove|            features|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+
|15.26|    14.84|      0.871|             5.763|             3.312|                2.221|              5.22|[15.26,14.84,0.87...|
|14.88|    14.57|     0.8811| 5.553999999999999|             3.333|                1.018|             4.956|[14.88,14.57,0.88...|
|14.29|    14.09|      0.905|             5.291|3.3369999999999997|                2.699|             4.825|[14.29,14.09,0.90...|
|13.84|    13.94|     0.8955|             5.324|3.3789999999999996|                2.259|             4.805|[13.84,13.94,0.89...|
|16.14|    14.99|     0.9034|5.6579999999999995|             3.562|                1.355| 

In [13]:
from pyspark.ml.feature import StandardScaler

In [15]:
scaler = StandardScaler(inputCol='features',
                        outputCol='scaledFeatures')

In [16]:
scaler_model = scaler.fit(final)

In [17]:
final = scaler_model.transform(final)

In [19]:
final.head(1)

[Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22, features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]), scaledFeatures=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621]))]

In [20]:
kmean = KMeans(featuresCol='scaledFeatures',k=3)

In [22]:
model = kmean.fit(final)

In [23]:
model.clusterCenters

<bound method KMeansModel.clusterCenters of KMeansModel: uid=KMeans_62644a06a9e4, k=3, distanceMeasure=euclidean, numFeatures=7>

In [25]:
model.transform(final).select('scaledFeatures','prediction').show()

+--------------------+----------+
|      scaledFeatures|prediction|
+--------------------+----------+
|[5.24452795332028...|         0|
|[5.11393027165175...|         0|
|[4.91116018695588...|         0|
|[4.75650503761158...|         0|
|[5.54696468981581...|         0|
|[4.94209121682475...|         0|
|[5.04863143081749...|         0|
|[4.84929812721816...|         0|
|[5.71536696354628...|         2|
|[5.65006812271202...|         2|
|[5.24452795332028...|         0|
|[4.82180387844584...|         0|
|[4.77368894309428...|         0|
|[4.73588435103234...|         0|
|[4.72213722664617...|         0|
|[5.01426361985209...|         0|
|[4.80805675405968...|         0|
|[5.39230954047151...|         0|
|[5.05206821191403...|         0|
|[4.37158555479908...|         1|
+--------------------+----------+
only showing top 20 rows

